# Exercise 3 : Sensitivity analysis
Preforming sensitivity analysis on the TV production LP-problem from Exercise 1.

## Imports

In [18]:
import numpy as np
from scipy.optimize import linprog

## Setup

In [19]:
# Original problem from Exercise 1
print("\nOriginal Problem Setup:")
c = np.array([700, 1000])  # Objective: profit per TV type A and B
A = np.array([[3, 5], [1, 3], [2, 2]])  # Stage I  # Stage II      # Stage III
b = np.array([3900, 2100, 2200])

print(f"Objective coefficients: c = {c}")
print(f"Constraint matrix A =\n{A}")
print(f"Right-hand side b = {b}")

# Solve the primal problem
result_primal = linprog(
    -c, A_ub=A, b_ub=b, bounds=[(0, None), (0, None)], method="highs"
)
print(f"\nPrimal optimal solution: x* = {result_primal.x}")
print(f"Primal optimal value: z* = {-result_primal.fun}")


Original Problem Setup:
Objective coefficients: c = [ 700 1000]
Constraint matrix A =
[[3 5]
 [1 3]
 [2 2]]
Right-hand side b = [3900 2100 2200]

Primal optimal solution: x* = [800. 300.]
Primal optimal value: z* = 860000.0


## i) Formulate and solve the dual problem.

In [20]:
print(
    """
Primal problem:
  max   700*x1 + 1000*x2
  s.t   3*x1 + 5*x2 <= 3900  (Stage I)
        1*x1 + 3*x2 <= 2100  (Stage II)
        2*x1 + 2*x2 <= 2200  (Stage III)
        x1, x2 >= 0

Dual problem:
  min   3900*y1 + 2100*y2 + 2200*y3
  s.t   3*y1 + 1*y2 + 2*y3 >= 700   (for x1)
        5*y1 + 3*y2 + 2*y3 >= 1000  (for x2)
        y1, y2, y3 >= 0

where y1, y2, y3 are the dual variables (shadow prices).
"""
)

# Dual problem matrices
c_dual = b  # Dual objective is primal RHS
A_dual = A.T  # Dual constraints are transpose of primal
b_dual = c  # Dual RHS is primal objective

# Solve dual (note: dual is minimization with >= constraints, so we use -A_ub)
result_dual = linprog(
    c_dual, A_ub=-A_dual, b_ub=-b_dual, bounds=[(0, None)] * 3, method="highs"
)

print(f"Dual optimal solution: y* = {result_dual.x}")
print(f"Dual optimal value: w* = {result_dual.fun}")
print("\nVerification:")
print(f"    Primal optimal value: z* = {-result_primal.fun:.2f}")
print(f"    Dual optimal value:   w* = {result_dual.fun:.2f}")
print(f"    Strong duality holds: {np.isclose(-result_primal.fun, result_dual.fun)}")


Primal problem:
  max   700*x1 + 1000*x2
  s.t   3*x1 + 5*x2 <= 3900  (Stage I)
        1*x1 + 3*x2 <= 2100  (Stage II)
        2*x1 + 2*x2 <= 2200  (Stage III)
        x1, x2 >= 0

Dual problem:
  min   3900*y1 + 2100*y2 + 2200*y3
  s.t   3*y1 + 1*y2 + 2*y3 >= 700   (for x1)
        5*y1 + 3*y2 + 2*y3 >= 1000  (for x2)
        y1, y2, y3 >= 0

where y1, y2, y3 are the dual variables (shadow prices).

Dual optimal solution: y* = [150.   0. 125.]
Dual optimal value: w* = 860000.0000000001

Verification:
    Primal optimal value: z* = 860000.00
    Dual optimal value:   w* = 860000.00
    Strong duality holds: True


## ii) Compute shadow prices.

In [21]:
shadow_prices = result_dual.x
print(f"Shadow prices (dual variables): y* = {shadow_prices}")
print(f"    Stage I   (y1): {shadow_prices[0]:.4f}")
print(f"    Stage II  (y2): {shadow_prices[1]:.4f}")
print(f"    Stage III (y3): {shadow_prices[2]:.4f}")

print(
    """
Interpretation:
- Shadow price represents the rate of change in the optimal objective
  value per unit increase in the resource availability.
- A shadow price of 0 indicates the constraint is not binding (there's slack).
- Higher shadow prices indicate more valuable resources.
"""
)

Shadow prices (dual variables): y* = [150.   0. 125.]
    Stage I   (y1): 150.0000
    Stage II  (y2): 0.0000
    Stage III (y3): 125.0000

Interpretation:
- Shadow price represents the rate of change in the optimal objective
  value per unit increase in the resource availability.
- A shadow price of 0 indicates the constraint is not binding (there's slack).
- Higher shadow prices indicate more valuable resources.



## iii) 100 extra working hours - where to invest?

In [22]:
print("\nTesting the impact of adding 100 hours to each stage:")

for i, stage_name in enumerate(["Stage I", "Stage II", "Stage III"]):
    b_modified = b.copy()
    b_modified[i] += 100

    result_modified = linprog(
        -c, A_ub=A, b_ub=b_modified, bounds=[(0, None), (0, None)], method="highs"
    )

    profit_increase = -result_modified.fun - (-result_primal.fun)
    predicted_increase = shadow_prices[i] * 100

    print(f"\n{stage_name}:")
    print(f"    Shadow price: {shadow_prices[i]:.4f}")
    print(f"    Predicted profit increase: {predicted_increase:.2f}")
    print(f"    Actual profit increase: {profit_increase:.2f}")
    print(f"    New optimal value: {-result_modified.fun:.2f}")

print(
    """
Recommendation:
- Invest in the stage with the highest shadow price for maximum benefit.
- Avoid investing in stages with shadow price of 0 (non-binding constraints).
"""
)

if shadow_prices[0] > shadow_prices[1] and shadow_prices[0] > shadow_prices[2]:
    print(" INVEST in Stage I ")
elif shadow_prices[1] > shadow_prices[0] and shadow_prices[1] > shadow_prices[2]:
    print(" INVEST in Stage II ")
else:
    print(" INVEST in Stage III ")

if np.min(shadow_prices) == 0:
    avoid_stage = np.argmin(shadow_prices)
    print(f" DO NOT invest in Stage {avoid_stage + 1} (shadow price = 0) ")


Testing the impact of adding 100 hours to each stage:

Stage I:
    Shadow price: 150.0000
    Predicted profit increase: 15000.00
    Actual profit increase: 15000.00
    New optimal value: 875000.00

Stage II:
    Shadow price: 0.0000
    Predicted profit increase: 0.00
    Actual profit increase: 0.00
    New optimal value: 860000.00

Stage III:
    Shadow price: 125.0000
    Predicted profit increase: 12500.00
    Actual profit increase: 12500.00
    New optimal value: 872500.00

Recommendation:
- Invest in the stage with the highest shadow price for maximum benefit.
- Avoid investing in stages with shadow price of 0 (non-binding constraints).

 INVEST in Stage I 
 DO NOT invest in Stage 2 (shadow price = 0) 


## iv) Price increase for type B to change optimal solution.

In [23]:
print(f"\nCurrent optimal solution: x* = {result_primal.x}")
print(f"Current price for type B: {c[1]}")

print("\nTesting different price increases for type B:")

# Test increasing prices
for price_increase in range(0, 300, 1):
    c_modified = c.copy()
    c_modified[1] = c[1] + price_increase

    result_test = linprog(
        -c_modified, A_ub=A, b_ub=b, bounds=[(0, None), (0, None)], method="highs"
    )

    print(
        f"  Price B = {c_modified[1]:4d}: x* = [{result_test.x[0]:6.1f}, {result_test.x[1]:6.1f}], z* = {-result_test.fun:7.1f}"
    )

    # Check if solution changed significantly
    if not np.allclose(result_test.x, result_primal.x, atol=1):
        print(f"\n Optimal solution changes at price increase of {price_increase} ")
        print(f"New optimal solution: x* = {result_test.x}")
        price_change_threshold = price_increase
        break


Current optimal solution: x* = [800. 300.]
Current price for type B: 1000

Testing different price increases for type B:
  Price B = 1000: x* = [ 800.0,  300.0], z* = 860000.0
  Price B = 1001: x* = [ 800.0,  300.0], z* = 860300.0
  Price B = 1002: x* = [ 800.0,  300.0], z* = 860600.0
  Price B = 1003: x* = [ 800.0,  300.0], z* = 860900.0
  Price B = 1004: x* = [ 800.0,  300.0], z* = 861200.0
  Price B = 1005: x* = [ 800.0,  300.0], z* = 861500.0
  Price B = 1006: x* = [ 800.0,  300.0], z* = 861800.0
  Price B = 1007: x* = [ 800.0,  300.0], z* = 862100.0
  Price B = 1008: x* = [ 800.0,  300.0], z* = 862400.0
  Price B = 1009: x* = [ 800.0,  300.0], z* = 862700.0
  Price B = 1010: x* = [ 800.0,  300.0], z* = 863000.0
  Price B = 1011: x* = [ 800.0,  300.0], z* = 863300.0
  Price B = 1012: x* = [ 800.0,  300.0], z* = 863600.0
  Price B = 1013: x* = [ 800.0,  300.0], z* = 863900.0
  Price B = 1014: x* = [ 800.0,  300.0], z* = 864200.0
  Price B = 1015: x* = [ 800.0,  300.0], z* = 864500.

## v) New TV type C.

In [24]:
print(
    """
Type C TV specifications:
- Profit: 1350
- Production times: (7, 4, 2) hours for stages I, II, III
"""
)

# Extended problem with type C
c_extended = np.array([700, 1000, 1350])  # Include type C
A_extended = np.array(
    [[3, 5, 7], [1, 3, 4], [2, 2, 2]]  # Stage I  # Stage II  # Stage III
)

# Solve extended problem
result_extended = linprog(
    -c_extended,
    A_ub=A_extended,
    b_ub=b,
    bounds=[(0, None), (0, None), (0, None)],
    method="highs",
)

print("\nSolution with type C included:")
print(f"    x* = {result_extended.x}")
print("    (x1=Type A, x2=Type B, x3=Type C)")
print(f"    Optimal value: z* = {-result_extended.fun:.2f}")

# Calculate reduced cost for type C
# Reduced cost formula: R_j = c_j - y* @ a_j
# where c_j is the objective coefficient, y* are shadow prices (dual variables),
# and a_j is the column of constraint coefficients for variable j
# A positive reduced cost indicates the variable should enter the basis (be produced)
c_C = 1350
a_C = np.array([7, 4, 2])
reduced_cost_C = c_C - shadow_prices @ a_C

print("\nReduced cost for Type C (from original problem):")
print(f"    c_C - y* @ a_C = {c_C} - {shadow_prices} @ {a_C}")
print(f"    = {c_C} - {shadow_prices @ a_C:.2f}")
print(f"    = {reduced_cost_C:.2f}")

print("\nInterpretation:")
if reduced_cost_C > 0:
    print(f" Type C should be PRODUCED (positive reduced cost = {reduced_cost_C:.2f}) ")
    print("The reduced cost indicates the net benefit of producing type C.")
else:
    print(" Type C should NOT be produced (reduced cost = {reduced_cost_C:.2f}) ")
    print("The resources would be better used for types A and B.")

print("\nVerification from optimization:")
if result_extended.x[2] > 0.1:
    print(
        f" The optimization confirms: Type C is produced ({result_extended.x[2]:.2f} units) "
    )
else:
    print(" The optimization confirms: Type C is not produced ")


Type C TV specifications:
- Profit: 1350
- Production times: (7, 4, 2) hours for stages I, II, III


Solution with type C included:
    x* = [950.   0. 150.]
    (x1=Type A, x2=Type B, x3=Type C)
    Optimal value: z* = 867500.00

Reduced cost for Type C (from original problem):
    c_C - y* @ a_C = 1350 - [150.   0. 125.] @ [7 4 2]
    = 1350 - 1300.00
    = 50.00

Interpretation:
 Type C should be PRODUCED (positive reduced cost = 50.00) 
The reduced cost indicates the net benefit of producing type C.

Verification from optimization:
 The optimization confirms: Type C is produced (150.00 units) 


## vi) Quality inspection working hours.

In [25]:
print(
    """
Quality inspection times:
- Type A: 0.5 hours
- Type B: 0.75 hours
- Type C: 0.1 hours (6 minutes)
"""
)

# Use the optimal production amounts to calculate inspection hours
optimal_production = result_extended.x
inspection_times = np.array([0.5, 0.75, 0.1])

total_inspection_hours = optimal_production @ inspection_times

print(f"\nOptimal production amounts: {optimal_production}")
print(f"Inspection times: {inspection_times} hours")
print(f"Total inspection hours needed: {total_inspection_hours:.2f} hours")

print(
    f"""
To maintain the current optimal production without interference:
 The company must add {total_inspection_hours:.2f} hours to the quality inspection line
"""
)


Quality inspection times:
- Type A: 0.5 hours
- Type B: 0.75 hours
- Type C: 0.1 hours (6 minutes)


Optimal production amounts: [950.   0. 150.]
Inspection times: [0.5  0.75 0.1 ] hours
Total inspection hours needed: 490.00 hours

To maintain the current optimal production without interference:
 The company must add 490.00 hours to the quality inspection line

